# load pkg

In [1]:
import torch
from transformers import AutoModelForCausalLM,AutoTokenizer
from trl import DPOConfig,DPOTrainer
from datasets import load_dataset

[2025-03-01 20:46:12,431] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/mscer/workingspace/anaconda3/envs/handson_llm/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -laio: 没有那个文件或目录
collect2: error: ld returned 1 exit status
/home/mscer/workingspace/anaconda3/envs/handson_llm/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -lcufile: 没有那个文件或目录
collect2: error: ld returned 1 exit status


# load model

In [2]:
device = 'cuda'

In [3]:
#AutoModelForCausalLM.from_pretrained??

In [4]:
model = AutoModelForCausalLM.from_pretrained("../models/Qwen/Qwen2.5-1.5B-Instruct/",torch_dtype=torch.float16,load_in_4bit=True).to(device)
tokenizer = AutoTokenizer.from_pretrained("../models/Qwen/Qwen2.5-1.5B-Instruct/")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [5]:
from peft import get_peft_model, LoraConfig,TaskType

In [6]:
TaskType?

Init signature:
TaskType(
    value,
    names=None,
    *,
    module=None,
    qualname=None,
    type=None,
    start=1,
)
Docstring:     
Enum class for the different types of tasks supported by PEFT.

Overview of the supported task types:
- SEQ_CLS: Text classification.
- SEQ_2_SEQ_LM: Sequence-to-sequence language modeling.
- CAUSAL_LM: Causal language modeling.
- TOKEN_CLS: Token classification.
- QUESTION_ANS: Question answering.
- FEATURE_EXTRACTION: Feature extraction. Provides the hidden states which can be used as embeddings or features
  for downstream tasks.
File:           ~/workingspace/anaconda3/envs/handson_llm/lib/python3.10/site-packages/peft/utils/peft_types.py
Type:           EnumMeta
Subclasses:     

In [7]:
lora_config = LoraConfig(task_type=TaskType.CAUSAL_LM,r = 8,lora_alpha=8)

In [8]:
get_peft_model?

Signature:
get_peft_model(
    model: 'PreTrainedModel',
    peft_config: 'PeftConfig',
    adapter_name: 'str' = 'default',
    mixed: 'bool' = False,
    autocast_adapter_dtype: 'bool' = True,
    revision: 'Optional[str]' = None,
) -> 'PeftModel | PeftMixedModel'
Docstring:
Returns a Peft model object from a model and a config.

Args:
    model ([`transformers.PreTrainedModel`]):
        Model to be wrapped.
    peft_config ([`PeftConfig`]):
        Configuration object containing the parameters of the Peft model.
    adapter_name (`str`, `optional`, defaults to `"default"`):
        The name of the adapter to be injected, if not provided, the default adapter name is used ("default").
    mixed (`bool`, `optional`, defaults to `False`):
        Whether to allow mixing different (compatible) adapter types.
    autocast_adapter_dtype (`bool`, *optional*):
        Whether to autocast the adapter dtype. Defaults to `True`. Right now, this will only cast adapter weights
        using flo

In [9]:
lora_model = get_peft_model(model=model,peft_config=lora_config,mixed=False)

# train by dpoTrainer

## data process

In [10]:
load_dataset?

Signature:
load_dataset(
    path: str,
    name: Optional[str] = None,
    data_dir: Optional[str] = None,
    data_files: Union[str, Sequence[str], Mapping[str, Union[str, Sequence[str]]], NoneType] = None,
    split: Union[str, datasets.splits.Split, NoneType] = None,
    cache_dir: Optional[str] = None,
    features: Optional[datasets.features.features.Features] = None,
    download_config: Optional[datasets.download.download_config.DownloadConfig] = None,
    download_mode: Union[datasets.download.download_manager.DownloadMode, str, NoneType] = None,
    verification_mode: Union[datasets.utils.info_utils.VerificationMode, str, NoneType] = None,
    keep_in_memory: Optional[bool] = None,
    save_infos: bool = False,
    revision: Union[str, datasets.utils.version.Version, NoneType] = None,
    token: Union[bool, str, NoneType] = None,
    streaming: bool = False,
    num_proc: Optional[int] = None,
    storage_options: Optional[Dict] = None,
    trust_remote_code: bool = None,
   

In [11]:
data_path = '../data/ultrafeedback_binarized/'
dataset = load_dataset(path=data_path)

In [12]:
dataset.keys()

dict_keys(['train_prefs', 'train_sft', 'test_prefs', 'test_sft', 'train_gen', 'test_gen'])

In [13]:
len(dataset['train_prefs'])

61135

## trainer 

In [14]:
DPOConfig?

Init signature:
DPOConfig(
    output_dir: str,
    overwrite_output_dir: bool = False,
    do_train: bool = False,
    do_eval: bool = False,
    do_predict: bool = False,
    eval_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no',
    prediction_loss_only: bool = False,
    per_device_train_batch_size: int = 8,
    per_device_eval_batch_size: int = 8,
    per_gpu_train_batch_size: Optional[int] = None,
    per_gpu_eval_batch_size: Optional[int] = None,
    gradient_accumulation_steps: int = 1,
    eval_accumulation_steps: Optional[int] = None,
    eval_delay: Optional[float] = 0,
    torch_empty_cache_steps: Optional[int] = None,
    learning_rate: float = 1e-06,
    weight_decay: float = 0.0,
    adam_beta1: float = 0.9,
    adam_beta2: float = 0.999,
    adam_epsilon: float = 1e-08,
    max_grad_norm: float = 1.0,
    num_train_epochs: float = 3.0,
    max_steps: int = -1,
    lr_scheduler_type: Union[transformers.trainer_utils.SchedulerType, str] = 'linear',

### set dpo_config

In [15]:
dpo_config = DPOConfig(output_dir='./test_dpo_trainer/',do_train=True,do_eval=True,learning_rate=5.0e-7,\
                       num_train_epochs=1,per_device_train_batch_size=2,per_device_eval_batch_size=4,gradient_checkpointing=True,\
                       logging_steps=1,eval_strategy='steps',eval_steps=10)

In [16]:
DPOTrainer?

Init signature:
DPOTrainer(
    model: Union[transformers.modeling_utils.PreTrainedModel, torch.nn.modules.module.Module, str, NoneType] = None,
    ref_model: Union[transformers.modeling_utils.PreTrainedModel, torch.nn.modules.module.Module, str, NoneType] = None,
    args: Optional[trl.trainer.dpo_config.DPOConfig] = None,
    data_collator: Optional[transformers.data.data_collator.DataCollator] = None,
    train_dataset: Optional[datasets.arrow_dataset.Dataset] = None,
    eval_dataset: Union[datasets.arrow_dataset.Dataset, dict[str, datasets.arrow_dataset.Dataset], NoneType] = None,
    processing_class: Union[transformers.tokenization_utils_base.PreTrainedTokenizerBase, transformers.image_processing_utils.BaseImageProcessor, transformers.feature_extraction_utils.FeatureExtractionMixin, transformers.processing_utils.ProcessorMixin, NoneType] = None,
    model_init: Optional[Callable[[], transformers.modeling_utils.PreTrainedModel]] = None,
    compute_metrics: Optional[Callable[[tr

In [17]:
lora_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear4bit(in_features=153

In [19]:
dpo_trainer = DPOTrainer(model=lora_model,ref_model=None,args = dpo_config,\
                         train_dataset=dataset['train_prefs'].remove_columns('messages'),
                         eval_dataset=dataset['test_prefs'].remove_columns('messages'),
                         processing_class=tokenizer,
                         )

zjq_debug2l {'writer_batch_size': 10, 'num_proc': None, 'desc': 'Extracting prompt in train dataset'} {'prompt': [{'content': 'how can i develop a habit of drawing daily', 'role': 'user'}], 'prompt_id': '086b3e24f29b8956a01059f79c56db35d118a06fb6b844b095737d042795cd43', 'chosen': [{'content': "Developing a daily habit of drawing can be challenging but with consistent practice and a few tips, it can become an enjoyable and rewarding part of your daily routine. Here are some strategies to help you develop the habit of drawing daily:\n\n1. Set a specific time: Allocate a specific time of the day to draw. It could be in the morning, afternoon, or evening. Make drawing a part of your daily routine.\n2. Set a specific duration: Determine the amount of time you want to spend on drawing each day. It can be as little as 10 minutes or as long as an hour. Be consistent with the duration to help build the habit.\n3. Start small and simple: Don't try to create a masterpiece every day, start with si

In [ ]:
dpo_trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jiaqi-zjq to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/home/mscer/workingspace/anaconda3/envs/handson_llm/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
/home/mscer/workingspace/anaconda3/envs/handson_llm/lib/python3.10/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/chosen,Logps/rejected,Logits/chosen,Logits/rejected
10,0.688500,0.692871,0.000753,0.000060,0.319500,0.000693,-361.091370,-324.491638,-0.774851,-0.697901
